In [0]:
%tensorflow_version 2.x

In [0]:
import pandas as pd
import random
import math
from tqdm import trange
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from google.colab import drive

In [3]:
###Mount drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
### custom Layer
class Embbeding(tf.keras.layers.Layer):
  
  def __init__(self,len_onehot_u,len_onehot_i):
    self.len_onehot_u=len_onehot_u
    self.len_onehot_i=len_onehot_i
    super(Embbeding,self).__init__()

  def build(self,input_shape=(0,0)):
    self.mat_emb_u=self.add_weight("embbed_transform_matrix_u",shape=[256,self.len_onehot_u],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
    self.mat_emb_i=self.add_weight("embbed_tranform_matrix_i",shape=[256,self.len_onehot_i],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
    self.bias_u=self.add_weight("bias_u",shape=[256,],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
    self.bias_i=self.add_weight("bias_i",shape=[256,],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
  
  def call(self,input):
    user_onehot,item_onehot=input
    return tf.transpose(tf.stack([tf.add(tf.linalg.matvec(self.mat_emb_u,user_onehot),self.bias_u),tf.add(tf.linalg.matvec(self.mat_emb_i,item_onehot),self.bias_i)]))

class Dense(tf.keras.layers.Layer):

  def __init__(self, reduc_to,len_input):
    self.reduc_to=reduc_to
    self.len_input=len_input
    super(Dense,self).__init__()

  def build(self,input_shape=(0,0)):
    self.mat_reduc=self.add_weight("reduc_mat",shape=[self.reduc_to,self.len_input],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
    self.bias=self.add_weight("bias",shape=[self.reduc_to,1],dtype=tf.float32,initializer=tf.initializers.RandomUniform)

  def call(self,input):
      return tf.add(tf.matmul(self.mat_reduc,input),self.bias)
  

class Linear(tf.keras.layers.Layer):

  def __init__(self,len_input):
    self.len_input=len_input
    super(Linear,self).__init__()

  def build(self,input_shape=(20000,1)):
    self.mat_reduc=self.add_weight("reduc_mat",shape=[2,1],dtype=tf.float32,initializer=tf.initializers.RandomUniform)
    self.bias=self.add_weight("bias",shape=[self.len_input,1],dtype=tf.float32, initializer=tf.initializers.RandomUniform)

  def call(self,input):
    return tf.add(tf.matmul(input, self.mat_reduc),self.bias)

class DeepRs(tf.keras.Model):

  def __init__(self,len_onehot_u,len_onehot_i):
    super(DeepRs,self).__init__()
    self.step_func=tf.keras.activations.relu
    self.layer1=Embbeding(len_onehot_u,len_onehot_i)
    self.layer2=Linear(256)
    self.layer3=Dense(128,256)
    self.layer4=Dense(32,128)
    self.layer5=Dense(1,32)

  def call(self,input):
    output=self.layer1(input)
    output=self.step_func(output)
    output=self.layer2(output)
    output=self.step_func(output)
    output=self.layer3(output)
    output=self.step_func(output)
    output=self.layer4(output)
    output=self.step_func(output)
    output=self.layer5(output)
    output=self.step_func(output)
    return output[0][0]

## cost func
def cost_fn(source, target):
  diff=source-target
  diff_sqr=tf.square(diff)
  return diff_sqr

In [0]:
### Load Data
train=pd.read_csv("/content/drive/Shared drives/Chula Drive/Datasets/AnimeRec-w  review/train_expli_matrix")
train=train.set_index("uid")
test=pd.read_csv("/content/drive/Shared drives/Chula Drive/Datasets/AnimeRec-w  review/test_expli_matrix")
test=test.set_index("uid")

In [6]:
train_set_input=[]
train_set_output=[]
for i in trange(28555):
  voted=np.where(train.iloc[[i],:].values[0]>0)[0]
  for j in voted:
    user_onehot=tf.one_hot(i,28555,dtype=tf.float32)
    item_onehot=tf.one_hot(j,3552,dtype=tf.float32)
    train_set_input.append(tuple((user_onehot,item_onehot)))
    train_set_output.append(train.iloc[[i],[j]].values[0][0])

for i in trange(28555):
  unvoted=np.where(train.iloc[[i],:].values[0]==0)[0]
  voted=np.where(train.iloc[[i],:].values[0]>0)[0]
  for j in range(math.ceil(len(voted)*0.25)):
    user_onehot=tf.one_hot(i,28555,dtype=tf.float32)
    item_onehot=tf.one_hot(unvoted[random.randrange(len(unvoted)-1)],3552,dtype=tf.float32)
    train_set_input.append(tuple((user_onehot,item_onehot)))
    train_set_output.append(train.iloc[[i],[j]].values[0][0])

100%|██████████| 28555/28555 [01:18<00:00, 365.92it/s]


In [0]:
model=DeepRs(28555,3552)

In [0]:
### save to plot
losses=[]
steps=[]

### train (trange is just range with progress bar from tqdm module)

optimizer=tf.optimizers.Adam(learning_rate=0.05)
for i in trange(5, desc="epoch"):
  for j in range(len(train_set_input)):
    with tf.GradientTape() as tape:
      predicted=model(train_set_input[j])
      loss=cost_fn(train_set_output[j],predicted)
      losses.append(loss.numpy())
      steps.append(i)
    variables=model.trainable_weights
    gradient=tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(gradient,variables))

# plot training loss
plt.plot(steps, losses, marker="o")
plt.xlabel("steps")
plt.ylabel("loss")
plt.show()

epoch:   0%|          | 0/5 [00:00<?, ?it/s]